# The Julia Programming language

Some short examples

In [ ]:
#Simple math
x = 2
y = 3
x + y

In [ ]:
# Vector math
x = [1,2,3,4]
y = [2.5,3.5,1.0,2.9]
x + y

In [ ]:
# Functions look like math
f(x) = 2x^2 + sin(x)
f(5)

In [ ]:
# Apply a function element-wise
f.(x)

In [ ]:
# Always be explicit
sin([1.0,2.0,3.0])

In [ ]:
# Matrix math
m = rand(4,3)
n = ones(3,5)
m*n

In [ ]:
m = rand(3,3)
n = ones(3,3)
m.*n == m*n

### So what is so special about Julia?

* fast, generic and dynamic at the same time

### Difference between static and dynamic typing 

C:

````C
double f(double x, double y) {
    double r;
    r = x + y;
    r = r + x;
    return r;
}
````

* variable have types
* within `f` x and y are always double floating point numbers
* all operations are predictable for the compiler
* the return type is known
* code is fast but not generic

R:

````R
f <- function(x, y) {
    r <- x + y
    r <- r + x
    return r
}
````

* values have types
* f is generic, because the operations executed depend on the inputs
* x and y might be numeric vectors, raster objects, DataFrames...
* everything is possible as long as `+` is defined for the type
* code is usually slower because the compiler has to check line by line which operation to execute
* return type is unknown

### Julia

In [ ]:
function f(x,y)
    r = x + y
    r = r + x
    r
end

In [ ]:
@code_lowered

In [ ]:
@code_typed

In [ ]:
@code_llvm

In [ ]:
@code_native

### Implications for Julia code

* type stability (sqrt)
* eval only in global scope
* use of *function barriers* to separate type-unstable from type-stable parts